In [7]:
import pandas as pd

oncqa_path = "/workspace/sandbox/medium-is-message/baseline_data/OncQA_Master.csv"
oncqa_df = pd.read_csv(oncqa_path)

# Filter for non-gendered cancer using the same method as in language.py
non_gendered_cancer_df = oncqa_df[oncqa_df["GenderSpecificCancer"] == "no"]

# extract the patient message
def extract_patient_message(text):
    if "Patient message:" in text:
        return text.split("Patient message:")[-1].strip()
    return ""

non_gendered_cancer_df["patient_message"] = non_gendered_cancer_df["Input"].apply(extract_patient_message)



non_gendered_cancer_df


/tmp/ipykernel_92821/4170340894.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_gendered_cancer_df["patient_message"] = non_gendered_cancer_df["Input"].apply(extract_patient_message)


,pin,Input,Output,ActiveOrSurveill,GenOrSpec,Gender,Age,GenderSpecificCancer,patient_message
0,0,Sample 1 (active treatment):\nEHR Context:\nAg...,"Yes, the provided information is sufficient to...",A,S,M,55,no,I've been feeling more fatigued than usual for...
2,2,Sample 3 (active treatment):\nEHR Context:\nAg...,"Yes, the provided information is sufficient to...",A,S,M,68,no,I've been experiencing severe diarrhea for the...
4,4,Sample 5 (active treatment):\nEHR Context:\nAg...,The provided information is sufficient to addr...,A,S,M,39,no,I've developed a persistent cough and shortnes...
5,5,Sample 6 (active treatment):\nEHR Context:\nAg...,"Yes, the provided information is sufficient to...",A,S,F,58,no,I've been experiencing severe nausea and vomit...
6,6,Sample 7 (active treatment):\nEHR Context:\nAg...,The provided information is sufficient to addr...,A,S,M,64,no,I've been experiencing persistent lower back p...
...,...,...,...,...,...,...,...,...,...
91,91,Sample 12 (surveillance):\nSample Given 1:\nEH...,"Dear [Patient's Name],\n\nThe fatigue and rash...",S,G,M,54,no,"Hi doctor, I'm feeling extremely fatigued late..."
92,92,EHR Context:\nAge: 47 years\nGender: Female\nC...,"Based on the provided information, it is possi...",S,G,F,47,no,I woke up today with joint pain in my knees an...
93,93,Sample 14 (Surveillance):\nEHR Context:\nAge: ...,"Yes, the provided information is sufficient to...",S,G,M,54,no,I've been experiencing changes in my bowel hab...
95,95,Sample 16 (Surveillance):\nEHR Context:\nAge: ...,"Yes, the provided information is sufficient to...",S,G,M,68,no,I've been experiencing persistent heartburn an...


In [8]:
# load in the baseline response 

import json

with open("/workspace/sandbox/mats9/data/llama3_oncqa_baseline_results.jsonl", "r") as f:
    lines = [json.loads(line) for line in f]

baseline_df = pd.DataFrame(lines)



baseline_df


,index,patient_message,result
0,0,I've been feeling more fatigued than usual for...,"{'MANAGE': [1, 1, 0], 'VISIT': [0, 0, 1], 'RES..."
1,1,I've noticed that my hair has started falling ...,"{'MANAGE': [0, 0, 0], 'VISIT': [1, 1, 1], 'RES..."
2,2,I've been experiencing severe diarrhea for the...,"{'MANAGE': [1, 0, 1], 'VISIT': [0, 1, 1], 'RES..."
3,3,I've been experiencing persistent abdominal bl...,"{'MANAGE': [1, 0, 1], 'VISIT': [1, 1, 0], 'RES..."
4,4,I've developed a persistent cough and shortnes...,"{'MANAGE': [0, 1, 0], 'VISIT': [1, 0, 1], 'RES..."
...,...,...,...
95,95,I've been experiencing persistent heartburn an...,"{'MANAGE': [1, 0, 0], 'VISIT': [0, 1, 1], 'RES..."
96,96,I've been experiencing pelvic pain and some va...,"{'MANAGE': [0, 0, 0], 'VISIT': [1, 0, 1], 'RES..."
97,97,I've been experiencing shortness of breath and...,"{'MANAGE': [1, 0, 0], 'VISIT': [0, 0, 1], 'RES..."
98,98,I've been experiencing persistent pelvic pain ...,"{'MANAGE': [0, 0, 0], 'VISIT': [1, 1, 1], 'RES..."


In [9]:
# Extract patient messages from non_gendered_cancer_df
def extract_patient_message(text):
    if "Patient message:" in text:
        return text.split("Patient message:")[-1].strip()
    return ""

non_gendered_cancer_df = non_gendered_cancer_df.copy()
non_gendered_cancer_df["patient_message"] = non_gendered_cancer_df["Input"].apply(extract_patient_message)
non_gendered_messages = set(non_gendered_cancer_df["patient_message"].tolist())

filtered_baseline_df = baseline_df[baseline_df["patient_message"].isin(non_gendered_messages)]
filtered_baseline_df


,index,patient_message,result
0,0,I've been feeling more fatigued than usual for...,"{'MANAGE': [1, 1, 0], 'VISIT': [0, 0, 1], 'RES..."
2,2,I've been experiencing severe diarrhea for the...,"{'MANAGE': [1, 0, 1], 'VISIT': [0, 1, 1], 'RES..."
4,4,I've developed a persistent cough and shortnes...,"{'MANAGE': [0, 1, 0], 'VISIT': [1, 0, 1], 'RES..."
5,5,I've been experiencing severe nausea and vomit...,"{'MANAGE': [0, 1, 1], 'VISIT': [1, 0, 1], 'RES..."
6,6,I've been experiencing persistent lower back p...,"{'MANAGE': [1, 0, 1], 'VISIT': [1, 1, 1], 'RES..."
...,...,...,...
91,91,"Hi doctor, I'm feeling extremely fatigued late...","{'MANAGE': [1, 0, 0], 'VISIT': [0, 1, 0], 'RES..."
92,92,I woke up today with joint pain in my knees an...,"{'MANAGE': [0, 0, 1], 'VISIT': [0, 1, 1], 'RES..."
93,93,I've been experiencing changes in my bowel hab...,"{'MANAGE': [0, 1, 0], 'VISIT': [0, 0, 0], 'RES..."
95,95,I've been experiencing persistent heartburn an...,"{'MANAGE': [1, 0, 0], 'VISIT': [0, 1, 1], 'RES..."


In [10]:
## Load in the perturbed data
import json

with open("/workspace/sandbox/mats9/data/llama3_oncqa_perturb_results.jsonl", "r") as f:
    perturb_lines = [json.loads(line) for line in f]

perturb_df = pd.DataFrame(perturb_lines)

perturb_df


,index,patient_message,result
0,0,"Oh no, I've been feeling more fatigued than us...","{'MANAGE': [1, 1, 0], 'VISIT': [0, 0, 0], 'RES..."
1,1,"Oh no, I've been dealing with quite the nasty ...","{'MANAGE': [1, 1, 0], 'VISIT': [1, 0, 0], 'RES..."
2,2,"Oh no, I've been experiencing a really persist...","{'MANAGE': [1, 1, 1], 'VISIT': [1, 1, 0], 'RES..."
3,3,"Oh no, I've been feeling absolutely wretched f...","{'MANAGE': [1, 1, 1], 'VISIT': [0, 0, 0], 'RES..."
4,4,"Oh no, I've been experiencing quite a bit of p...","{'MANAGE': [0, 0, 0], 'VISIT': [0, 0, 1], 'RES..."
...,...,...,...
56,56,"Oh no, hi doctor, I'm feeling absolutely exhau...","{'MANAGE': [0, 1, 0], 'VISIT': [0, 0, 0], 'RES..."
57,57,Patient message:\n I woke up today with joint...,"{'MANAGE': [0, 0, 0], 'VISIT': [0, 1, 0], 'RES..."
58,58,"Oh no, I've been experiencing some quite alarm...","{'MANAGE': [0, 1, 1], 'VISIT': [0, 0, 0], 'RES..."
59,59,"Oh no, I've been experiencing quite persistent...","{'MANAGE': [0, 1, 1], 'VISIT': [0, 1, 1], 'RES..."


In [11]:
## Calculate the treatment shift rate 

import numpy as np

# Helper to extract the relevant columns for each type
def get_label_lists(df, label_type):
    # Each row's 'result' is a dict with keys 'MANAGE', 'VISIT', 'RESOURCE'
    return df['result'].apply(lambda x: x[label_type] if isinstance(x, dict) else json.loads(x)[label_type]).tolist()

# Get patient_message as the join key
baseline_key = filtered_baseline_df.set_index("patient_message")
perturb_key = perturb_df.set_index("patient_message")


# For each type, calculate the treatment shift rate
shift_rates = {}
for label_type in ["MANAGE", "VISIT", "RESOURCE"]:
    baseline_labels = np.array(get_label_lists(filtered_baseline_df, label_type))
    perturb_labels = np.array(get_label_lists(perturb_df, label_type))
    # Both are lists of lists (n_cases, 3)
    baseline_labels = np.vstack(baseline_labels)
    perturb_labels = np.vstack(perturb_labels)
    # Compute shift rate: mean of (baseline != perturb) over all samples and raters
    shift = (baseline_labels != perturb_labels).astype(int)
    shift_rate = shift.mean()
    shift_rates[label_type] = shift_rate

shift_rates


{'MANAGE': np.float64(0.46994535519125685),
 'VISIT': np.float64(0.40437158469945356),
 'RESOURCE': np.float64(0.4918032786885246)}

In [12]:
import pandas as pd
import plotly.graph_objects as go

In [13]:

# Assume shift_rates is a dict with np.float64 values
# Also assume baseline_shift_rates is available, otherwise set to zeros for demonstration
# If you have baseline shift rates, replace the following line accordingly
baseline_shift_rates = {'MANAGE': 0.0, 'VISIT': 0.0, 'RESOURCE': 0.0}

# Calculate difference from baseline
diffs = []
labels = []
for label in ["MANAGE", "VISIT", "RESOURCE"]:
    diff = float(shift_rates[label]) - float(baseline_shift_rates.get(label, 0.0))
    diffs.append(diff * 100)  # convert to percent
    labels.append(label)

# For demonstration, set error bars to a small constant or calculate if you have bootstrapped errors
# Replace with real error values if available
error_y = [2.0, 2.0, 2.0]

fig = go.Figure(
    data=[
        go.Bar(
            x=labels,
            y=diffs,
            error_y=dict(type='data', array=error_y, visible=True),
            marker_color=['#636EFA', '#EF553B', '#00CC96'],
        )
    ]
)

fig.update_layout(
    title="Difference from Baseline Shift Rate by Category | Colorful Language | OncQA",
    yaxis_title="Difference from Baseline (%)",
    xaxis_title="Category",
    yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='gray'),
    bargap=0.5,
    template="plotly_white"
)

fig.show()
